# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto baquerizo moreno,-0.9000,-89.6000,24.55,61,75,6.17,EC,1730744648
1,1,olonkinbyen,70.9221,-8.7187,1.05,99,100,7.72,SJ,1730744649
2,2,luwuk,-0.9516,122.7875,22.28,83,94,1.96,ID,1730744651
3,3,san antonio de pale,-1.4014,5.6325,25.57,83,98,5.83,GQ,1730744652
4,4,ust-nera,64.5667,143.2000,-25.45,99,98,1.28,RU,1730744653


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    size='Humidity',  # Point size based on humidity
    color='Humidity',  # Color scale based on humidity
    cmap='viridis',
    alpha=0.6,
    geo=True,  # Enable geospatial plotting
    tiles='CartoLight',  # Background map tiles
    hover_cols=['City', 'Humidity']  # Display additional info on hover
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df[
    (city_data_df['Max Temp'] >= 25) & (city_data_df['Max Temp'] <= 33) & 
    (city_data_df['Wind Speed'] < 2.5)
]
# Drop any rows with null values
filtered_df = filtered_df.dropna()

# Display sample data
print(filtered_df)

     City_ID                 City      Lat       Lng  Max Temp  Humidity  \
15        15              bhakkar  31.6253   71.0657     25.92        21   
34        34               howrah  22.5892   88.3103     26.00        83   
48        48             lorengau  -2.0226  147.2712     26.59        77   
66        66                utrik  11.2278  169.8474     26.99        75   
69        69            taoudenni  22.6783   -3.9836     31.18        12   
81        81     kuala terengganu   5.3302  103.1408     25.97        89   
95        95           georgetown   5.4112  100.3354     26.96        83   
100      100       howard springs -12.4970  131.0470     25.95        90   
105      105            kavaratti  10.5669   72.6420     28.53        69   
124      124              mangrol  21.1167   70.1167     27.89        64   
131      131  san luis de la loma  17.2706 -100.8939     29.11        68   
146      146         shahr sultan  29.5765   71.0212     27.16        21   
172      172

### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
print(hotel_df)

     City_ID                 City      Lat       Lng  Max Temp  Humidity  \
15        15              bhakkar  31.6253   71.0657     25.92        21   
34        34               howrah  22.5892   88.3103     26.00        83   
48        48             lorengau  -2.0226  147.2712     26.59        77   
66        66                utrik  11.2278  169.8474     26.99        75   
69        69            taoudenni  22.6783   -3.9836     31.18        12   
81        81     kuala terengganu   5.3302  103.1408     25.97        89   
95        95           georgetown   5.4112  100.3354     26.96        83   
100      100       howard springs -12.4970  131.0470     25.95        90   
105      105            kavaratti  10.5669   72.6420     28.53        69   
124      124              mangrol  21.1167   70.1167     27.89        64   
131      131  san luis de la loma  17.2706 -100.8939     29.11        68   
146      146         shahr sultan  29.5765   71.0212     27.16        21   
172      172

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bhakkar - nearest hotel: No hotel found
howrah - nearest hotel: Sun India Guest House
lorengau - nearest hotel: Seeadler Bay Hotel
utrik - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
kuala terengganu - nearest hotel: Uncle Homestay
georgetown - nearest hotel: Page 63 hostel
howard springs - nearest hotel: Rydges Palmerston
kavaratti - nearest hotel: No hotel found
mangrol - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
shahr sultan - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
pasuruan - nearest hotel: Hotel Pasuruan
manokwari - nearest hotel: Hotel Arfak
tchollire - nearest hotel: No hotel found
badagry - nearest hotel: Kappo Hotel
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
jolo - nearest hotel: No hotel found
celestun - nearest hotel: Hotel Flamingo Playa
porangatu - nearest hotel: No hotel found
vanimo - nearest hotel: CBC Guesthouse
bandar abbas -

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,15,bhakkar,31.6253,71.0657,25.92,21,3,1.62,PK,1730744665,No hotel found
34,34,howrah,22.5892,88.3103,26.00,83,20,1.54,IN,1730744687,Sun India Guest House
48,48,lorengau,-2.0226,147.2712,26.59,77,100,0.58,PG,1730744703,Seeadler Bay Hotel
66,66,utrik,11.2278,169.8474,26.99,75,100,0.61,MH,1730744722,No hotel found
69,69,taoudenni,22.6783,-3.9836,31.18,12,4,1.99,ML,1730744727,No hotel found
81,81,kuala terengganu,5.3302,103.1408,25.97,89,20,1.03,MY,1730744740,Uncle Homestay
95,95,georgetown,5.4112,100.3354,26.96,83,20,2.06,MY,1730744756,Page 63 hostel
100,100,howard springs,-12.4970,131.0470,25.95,90,100,1.39,AU,1730744763,Rydges Palmerston
105,105,kavaratti,10.5669,72.6420,28.53,69,62,2.15,IN,1730744768,No hotel found
124,124,mangrol,21.1167,70.1167,27.89,64,0,2.03,IN,1730744789,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',
    size='Humidity',  # Point size based on humidity
    color='Humidity',  # Color scale based on humidity
    cmap='viridis',
    alpha=0.6,
    geo=True,  # Enable geospatial plotting
    tiles='CartoLight',  # Background map tiles
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']  # Display city, country, hotel, and humidity
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)